In [5]:
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver import ChromeOptions
from lxml import html
from selenium.webdriver.common.action_chains import ActionChains
import time
from selenium.webdriver.common.keys import Keys
import pandas as pd
import logging

import requests
from bs4 import BeautifulSoup
import json
import sys

In [6]:
def guarantee(driver,flag,df_flag,infoID):
    html=driver.page_source#获取网页的html数据
    soup=BeautifulSoup(html,'lxml')#对html进行解析
    table_select=soup.find('div',id = infoID)
    table = table_select.find('table',class_ = 'table table-bordered')
    name=[]
    try:
        for th in table.find_all('tr')[0].find_all('th'):
            name.append(th.get_text())#获取表格的字段名称作为字典的键
        #flag = 0 #标记，当爬取字段数据是为0，否则为1
        #df_flag = 0 #标记是否创建dataframe
        for tr in table.find_all('tr'):
        #第一行为表格字段数据，因此跳过第一行
            if flag==1:
                dic={}
                i=0
                for td in tr.find_all('td'):
                    dic[name[i]]=td.get_text().replace('\n','').replace('\r','')
                    i+=1
                if df_flag == 0:
                    df_guarantee = pd.DataFrame.from_dict(dic,orient='index').T
                    df_flag = 1
                else:
                    df_guarantee = df_guarantee.append([dic], ignore_index=True)    
            flag=1
        df_guarantee = df_guarantee.drop('详情',axis = 1)
    except:
        return False

    return df_guarantee,flag,df_flag

In [228]:
def guarantee(driver,df_guarantee, flag, df_flag):
    #driver.set_page_load_timeout(30)
    #time.sleep(3)
    #guarantee_page=driver.get('https://www.qixin.com/publicly/5db4a5ed-776b-46a5-a129-648a6f47de34')#使用get方法请求url，因为是模拟浏览器，所以不需要headers信息    
    html=driver.page_source#获取网页的html数据
    soup=BeautifulSoup(html,'lxml')#对html进行解析
    table_select=soup.find('div',id = 'companyEvents')
    table = table_select.find('table',class_ = 'table table-bordered')
    name=[]
    try:
        for th in table.find_all('tr')[0].find_all('th'):
            name.append(th.get_text())#获取表格的字段名称作为字典的键
    except IndexError:
        Error = '----------------------------------------------------------------'+'\n'+\
                '-------------There is No Guarantee Record Here------------------' +'\n'+\
                '----------------------------------------------------------------'
        return Error
    else:
        #flag = 0 #标记，当爬取字段数据是为0，否则为1
        #df_flag = 0 #标记是否创建dataframe
        for tr in table.find_all('tr'):
        #第一行为表格字段数据，因此跳过第一行
            if flag==1:
                dic={}
                i=0
                for td in tr.find_all('td'):
                    dic[name[i]]=td.get_text().replace('\n','').replace('\r','')
                    i+=1
                if df_flag == 0:
                    df_guarantee = pd.DataFrame.from_dict(dic,orient='index').T
                    df_flag = 1
                else:
                    df_guarantee = df_guarantee.append([dic], ignore_index=True)    
            flag=1
        df_guarantee = df_guarantee.drop('详情',axis = 1)
        return df_guarantee, flag, df_flag

In [20]:


def page1(driver,infoID):
    global df,flag,df_flag
    html_source=driver.page_source#获取网页的html数据
    tree = html.fromstring(html_source)
    
    try:
        len_page = len(tree.xpath('//*[@id='+'"'+str(infoID)+'"'+']/div[3]/div/nav/ul/li'))
        #len_page = len(tree.xpath('//*[@id='+'"'+str(infoID)+'"'+']/div[2]/div[2]/nav/ul/li'))
        //*[@id="companyEvents"]/div[3]/div/nav/ul/li[6]/a

        if len_page > 0:
            time.sleep(2)
            i = 2
            pre = 0
            flag = 0 #标记，当爬取字段数据是为0，否则为1
            df_flag = 0 #标记是否创建dataframe
            df = []
            while i<=len_page-1:
                print(i)
                print(len_page)
                logger.info('start track page')
                #b = driver.find_element_by_xpath('//*[@id="changeInfo"]/div[2]/div[2]/nav/ul/li[{}]/a'.format(i))
                b = driver.find_element_by_xpath('//*[@id='+'"'+str(infoID)+'"'+']/div[3]/div/nav/ul/li['+str(i)+']/a')
                logger.info('track successfully')
                cur = b.text 
                if int(cur) != int(pre) + 1:
                    #should_click = driver.find_element_by_xpath('//*[@id="changeInfo"]/div[2]/div[2]/nav/ul/li[5]/a')
                    should_click = driver.find_element_by_xpath('//*[@id='+'"'+str(infoID)+'"'+']/div[3]/div/nav/ul/li[5]/a')
                    cur = should_click.text
                    should_click.click()
                    logger.info('click 1 success')
                    time.sleep(3)
                    change = guarantee(driver,df, flag, df_flag)
                    logger.info('getdata success')
                    df = change[0]
                    flag = change[1]
                    df_flag = change[2]
                    pre = cur

                else:
                    driver.execute_script("arguments[0].click();", b)
                    logger.info('click 2 success')
                    time.sleep(3)
                    change = guarantee(driver,df, flag, df_flag)
                    df = change[0]
                    flag = change[1]
                    df_flag = change[2]
                    pre = cur
                    i += 1
                    

        else: 
            change = guarantee(driver,df, flag, df_flag)
            df = change[0]
            flag = change[1]
            df_flag = change[2]
    except:
        logger.error('cannot retrieve {} page for {}'.format(infoID,num))
        return False
    return df

In [8]:
def getData(driver,df, flag, df_flag, infoID):
    html=driver.page_source #获取网页的html数据
    soup=BeautifulSoup(html,'lxml') #对html进行解析
    table_select=soup.find('div',id = infoID)
    try:
        table = table_select.find('table',class_ = 'table table-bordered margin-t-1x')
        logger.info("track table success")
        name=[]
        for th in table.find_all('tr')[0].find_all('th'):
            name.append(th.get_text()) #获取表格的字段名称作为字典的键
        logger.info('get key name success')    
        
        for tr in table.find_all('tr'):
            if flag==1:
                dic={}
                i=0
                for td in tr.find_all('td'):
                    dic[name[i]]=td.get_text().replace('\n','')
                    i+=1
                if df_flag == 0:
                    df = pd.DataFrame.from_dict(dic,orient='index').T
                    df_flag = 1
                else:
                    df = df.append([dic], ignore_index=True)
            flag=1
        logger.info('get value success')
        if infoID == 'companyEvents':
            df = df.drop('详情',axis = 1)
        else:
            pass
    except:
        logger.error('cannot retrieve the {} data for {}'.format(infoID,num))
        pass
    return df,flag,df_flag

def page(driver,infoID):
    global df,flag,df_flag
    html_source=driver.page_source#获取网页的html数据
    tree = html.fromstring(html_source)
    
    try:
        
        len_page = len(tree.xpath('//*[@id='+'"'+str(infoID)+'"'+']/div[2]/div[2]/nav/ul/li'))

        if len_page > 0:
            time.sleep(2)
            i = 2
            pre = 0
            flag = 0 #标记，当爬取字段数据是为0，否则为1
            df_flag = 0 #标记是否创建dataframe
            df = []
            while i<=len_page-1:
                print(i)
                print(len_page)
                logger.info('start track page')
                #b = driver.find_element_by_xpath('//*[@id="changeInfo"]/div[2]/div[2]/nav/ul/li[{}]/a'.format(i))
                b = driver.find_element_by_xpath('//*[@id='+'"'+str(infoID)+'"'+']/div[2]/div[2]/nav/ul/li['+str(i)+']/a')
                logger.info('track successfully')
                cur = b.text 
                if int(cur) != int(pre) + 1:
                    #should_click = driver.find_element_by_xpath('//*[@id="changeInfo"]/div[2]/div[2]/nav/ul/li[5]/a')
                    should_click = driver.find_element_by_xpath('//*[@id='+'"'+str(infoID)+'"'+']/div[2]/div[2]/nav/ul/li[5]/a')
                    cur = should_click.text
                    should_click.click()
                    logger.info('click 1 success')
                    time.sleep(3)
                    change = getData(driver,df, flag, df_flag,infoID)
                    logger.info('getdata success')
                    df = change[0]
                    flag = change[1]
                    df_flag = change[2]
                    pre = cur

                else:
                    driver.execute_script("arguments[0].click();", b)
                    logger.info('click 2 success')
                    time.sleep(3)
                    change = getData(driver,df, flag, df_flag,infoID)
                    df = change[0]
                    flag = change[1]
                    df_flag = change[2]
                    pre = cur
                    i += 1
                    

        else: 
            change = getData(driver,df, flag, df_flag, infoID)
            df = change[0]
            flag = change[1]
            df_flag = change[2]
    except:
        logger.error('cannot retrieve {} page for {}'.format(infoID,num))
        return False
    return df

In [10]:
def changeInfoClean(data,num,name):
    data['变更前']=data['变更前'].str.replace('*','')#清除*
    data['变更后']=data['变更后'].str.replace('*','')
    for ind, row in data.iterrows():   
        try:
            if row['变更后'].find("新增") != -1:#检查变更后是否有新增标识
                continue
            elif ((row['变更事项'].find("董事")!= -1) #检查变更事项是否为董事、经理等
                or (row['变更事项'].find("高管") != -1)
                or (row['变更事项'].find("经理") != -1)
                or (row['变更事项'].find("法定代表人") != -1)
                or (row['变更事项'].find("负责人") != -1)
                or (row['变更事项'].find("监事") != -1)
                or (row['变更事项'].find("财务负责人") != -1)) :
                continue
            elif row['变更后'].find("姓名") != -1: #少数情况检查
                continue
            else:
                data = data.drop(index = ind) #删除其他行
        except:
            data = data.drop(index = ind) #删除其他行
   
    data = data.drop('序号',axis=1)#有序号一列可以删去
    data['证券代码'] = num
    data['证券简称'] = name
    order = ['证券代码','证券简称','变更日期','变更事项','变更前','变更后']
    data = data[order]
    data.to_csv('changeInfo.csv' ,encoding="utf_8_sig", mode='a',index=False, header= None) #输出到CSV,不覆盖

In [11]:
def lawSuitsClean(data,num,name):
    data.to_csv('lawSuits.csv',encoding="utf_8_sig", mode='a',index=False, header= None)

In [12]:
def companyEvents(data,num,name):
    data.to_csv('companyEvents.csv',encoding="utf_8_sig", mode='a',index=False, header= None)

In [13]:
def changeInfo(driver):
    driver.find_element_by_xpath('/html/body/div[2]/div[4]/div/div/div/div/ul/li[2]/a/h2').click()
    time.sleep(2)
    
    test_data = page(driver,infoID)
    try:
        changeInfoClean(test_data,num,name)
    except:
        logger.error('clean work failed')

In [14]:
def lawSuits(driver):
    driver.find_element_by_xpath('/html/body/div[2]/div[4]/div/div/div/div/ul/li[5]/a/h2').click()
    time.sleep(2)
    
    test_data = page(driver,infoID)
    try:
        lawSuitsClean(test_data,num,name)
    except:
        logger.error('clean work failed')

In [23]:
def companyEvents(driver):
    #driver.find_element_by_xpath('/html/body/div[2]/div[4]/div/div/div/div/ul/li[1]/a/h2').click()
    time.sleep(2)
    
    test_data = page1(driver,infoID)
    try:
        companyEventsClean(test_data,num,name)
    except:
        logger.error('clean work failed')
    

In [16]:
# Set up logger，if parameter is null return root logger
logger = logging.getLogger("debugInfo")
logger.setLevel(logging.INFO) 

# Set up handler
fh = logging.FileHandler("debug.log", mode='w', encoding="utf-8")
ch = logging.StreamHandler()

# Set format for logging output
formatter = logging.Formatter(
    fmt="%(asctime)s - %(name)s - %(filename)s - %(message)s",
    datefmt='%Y-%m-%d  %H:%M:%S %a')

# Set handler
fh.setFormatter(formatter)
ch.setFormatter(formatter)

# add handler to logger
logger.addHandler(fh)
logger.addHandler(ch)

In [17]:
infoID = input("Which table to update? changeInfo, lawSuits, companyEvents\n")

Which table to update? changeInfo, lawSuits, companyEvents
companyEvents


In [18]:
driver = webdriver.Chrome('chromedriver') 
driver.get('https://www.qixin.com/search?key=601628&page=1') 
time.sleep(2)

In [25]:
# 搜索公司
input_df = pd.read_csv('compy_lst_test.csv', header = 0)
ddf = input_df[["证券代码","证券简称"]]
for id, row in ddf.iterrows():
    num = str(row[0])
    name = row[1]
    

    #num='002348' 
    search_box = driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[1]/div/div[2]/div/div/div/div[1]/div[3]/span[1]/input[2]')
    search_box.send_keys(Keys.CONTROL, 'a')
    search_box.send_keys(num)
    search_box.send_keys(u'\ue007') 

    time.sleep(3)

    element = driver.find_element_by_css_selector('body > div.app-search-enterprise.bg-b6.padding-b-50px > div.container.margin-t-0-6x > div > div.col-md-18 > div.padding-h-1x.border-h-b4.border-t-b4.app-list-items > div.row.bg-white > div:nth-child(1) > div.col-2.clearfix > div.col-2-1 > div.company-title.font-18.font-f1 > a')
    driver.execute_script("arguments[0].click();", element)                             

    ###################爬数据##########################################################

    # browser换至当前页面
    search_window = driver.window_handles
    driver.switch_to_window(search_window[-1])

    # 工商变更
    try:
        driver.find_element_by_xpath('/html/body/div[5]/div[1]/div/div[1]').click()
        time.sleep(1)
    except:
        pass


    #driver.find_element_by_xpath('/html/body/div[2]/div[4]/div/div/div/div/ul/li[2]/a/h2').click()
    #time.sleep(2)
    if infoID == 'changeInfo':
        changeInfo(driver)
    elif infoID == 'lawSuits':
        lawSuits(driver)
    elif infoID == 'companyEvents':
        companyEvents(driver)
    else:
        print("input error")
    
    # 关闭当前页&回到搜索页面
    driver.close()
    time.sleep(1)
    window = driver.window_handles
    driver.switch_to_window(window[0])




<ipython-input-25-a5c30ada7c4a>:24: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(search_window[-1])
2020-08-24  09:36:33 Mon - debugInfo - <ipython-input-20-8a5be8d18c6f> - start track page
INFO:debugInfo:start track page
2020-08-24  09:36:33 Mon - debugInfo - <ipython-input-20-8a5be8d18c6f> - track successfully
INFO:debugInfo:track successfully
2020-08-24  09:36:33 Mon - debugInfo - <ipython-input-20-8a5be8d18c6f> - click 2 success
INFO:debugInfo:click 2 success


2
5


2020-08-24  09:36:36 Mon - debugInfo - <ipython-input-20-8a5be8d18c6f> - cannot retrieve companyEvents page for 600859
ERROR:debugInfo:cannot retrieve companyEvents page for 600859
2020-08-24  09:36:36 Mon - debugInfo - <ipython-input-23-bd21f2ac7ce7> - clean work failed
ERROR:debugInfo:clean work failed
<ipython-input-25-a5c30ada7c4a>:49: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(window[0])
2020-08-24  09:36:45 Mon - debugInfo - <ipython-input-20-8a5be8d18c6f> - cannot retrieve companyEvents page for 600861
ERROR:debugInfo:cannot retrieve companyEvents page for 600861
2020-08-24  09:36:45 Mon - debugInfo - <ipython-input-23-bd21f2ac7ce7> - clean work failed
ERROR:debugInfo:clean work failed
2020-08-24  09:36:56 Mon - debugInfo - <ipython-input-20-8a5be8d18c6f> - start track page
INFO:debugInfo:start track page
2020-08-24  09:36:56 Mon - debugInfo - <ipython-input-20-8a5be8d18c6f> - track successfully
INFO:debugInfo:track successfully
2020-08-24 

2
7


2020-08-24  09:36:59 Mon - debugInfo - <ipython-input-20-8a5be8d18c6f> - cannot retrieve companyEvents page for 600862
ERROR:debugInfo:cannot retrieve companyEvents page for 600862
2020-08-24  09:36:59 Mon - debugInfo - <ipython-input-23-bd21f2ac7ce7> - clean work failed
ERROR:debugInfo:clean work failed


KeyboardInterrupt: 

In [24]:
# 关闭当前页&回到搜索页面
driver.close()
window = driver.window_handles
driver.switch_to_window(window[-1])

<ipython-input-24-c4d3f5919f9e>:4: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(window[-1])


In [174]:
infoID

'companyEvents'

In [180]:
num='600859' 
name = 'qwe'
search_box = driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[1]/div/div[2]/div/div/div/div[1]/div[3]/span[1]/input[2]')
search_box.send_keys(Keys.CONTROL, 'a')
search_box.send_keys(num)
search_box.send_keys(u'\ue007') 

time.sleep(3)

element = driver.find_element_by_css_selector('body > div.app-search-enterprise.bg-b6.padding-b-50px > div.container.margin-t-0-6x > div > div.col-md-18 > div.padding-h-1x.border-h-b4.border-t-b4.app-list-items > div.row.bg-white > div:nth-child(1) > div.col-2.clearfix > div.col-2-1 > div.company-title.font-18.font-f1 > a')
driver.execute_script("arguments[0].click();", element)                             

###################爬数据##########################################################

# browser换至当前页面
search_window = driver.window_handles
driver.switch_to_window(search_window[-1])

# 工商变更
try:
    driver.find_element_by_xpath('/html/body/div[5]/div[1]/div/div[1]').click()
    time.sleep(1)
except:
    pass


#driver.find_element_by_xpath('/html/body/div[2]/div[4]/div/div/div/div/ul/li[2]/a/h2').click()
#time.sleep(2)
if infoID == 'changeInfo':
    changeInfo(driver)
elif infoID == 'lawSuits':
    lawSuits(driver)
elif infoID == 'companyEvents':
    companyEvents(driver)
else:
    print("input error")

# 关闭当前页&回到搜索页面
driver.close()
time.sleep(1)
window = driver.window_handles
driver.switch_to_window(window[0])


<ipython-input-180-e351725e8f9a>:17: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(search_window[-1])
2020-08-24  00:07:07 Mon - debugInfo - <ipython-input-177-e1cf1e546ef3> - track table success
2020-08-24  00:07:07 Mon - debugInfo - <ipython-input-177-e1cf1e546ef3> - cannot retrieve the companyEvents data for 600859
2020-08-24  00:07:07 Mon - debugInfo - <ipython-input-179-899bb5866bd2> - clean work failed
<ipython-input-180-e351725e8f9a>:42: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(window[0])


In [118]:
infoID

'changeInfo'